<a href="https://colab.research.google.com/github/ZAHID188/Q-learning_collab/blob/main/Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports


In [ ]:
import os
import numpy as np
import tensorflow as tf

#DeepQNetwork Class

learning rate =lr, number of action=n_actions,name= name of actions ,fcl_dim= fully connected layer dimension

In [ ]:
class deeepQNetwork(object):
  #learning rate =lr, number of action=n_actions,name= name of actions
  #fcl_dim= fully connected layer dimension,
  def __init__(self,lr,n_actions,name,fcl_dims=256,input_dims=(210,160,4),chkpt_dir='tmp/dqn'):
    self.lr=lr
    self.name=name
    self.n_actions=n_actions
    self.fcl_dims=fcl_dims
    self.input_dims=input_dims
    self.sess=tf.Session()
    self.build_network()
    self.sess.run(tf.global_variables_initializer())
    self.saver=tf.train.Saver()
    self.checkpoint_file=os.path.join(chkpt_dir,'deepqnet.ckpt')
    self.params=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope=self.name)


# Explanation of the Code Avobe

Sure, let's break down the code and explain each part:



> `class deeepQNetwork(object):`


This line defines a new class called deeepQNetwork which inherits from the base object class in Python.


> `def __init__(self, lr, n_actions, name, fcl_dims=256, input_dims=(210, 160, 4), chkpt_dir='tmp/dqn'):`



This is the constructor method` (__init__)` for the deeepQNetwork class. It initializes the object with the following parameters:



1.  `lr:` Learning rate for the neural network.
2. ` n_actions:` The number of possible actions in the environment.
3. `name:` A string representing the name of the network.
4. `fcl_dims=256:` The dimension of the fully connected layer in the neural network. The default value is set to 256.
5. `input_dims=(210, 160, 4):` The dimensions of the input data. In this case, it's a tuple of (210, 160, 4), which could represent the shape of an image or some other input data.
6. `chkpt_dir='tmp/dqn':` The directory path where the checkpoint files (model weights) will be saved or loaded from. The default value is set to 'tmp/dqn'.
> self.lr = lr
>
> self.name = name
>
>self.n_actions = n_actions
>
>self.fcl_dims = fcl_dims
>
>self.input_dims = input_dims
>
>self.sess = tf.Session()

>These lines store the parameter values as instance variables (self.lr, self.name, self.n_actions, self.fcl_dims, self.input_dims) and create a new TensorFlow session (self.sess).

# Build the net

In [1]:
  def build_net(self):
    with tf.variable_scope(self.name):
      self.input = tf.placeholder(tf.float32,
                                        shape=[None, *self.input_dims],
                                        name='inputs')
      self.actions = tf.placeholder(tf.float32,
                                          shape=[None, self.n_actions],
                                          name='action_taken')
      self.q_target = tf.placeholder(tf.float32,
                                           shape=[None, self.n_actions],
                                           name='q_value')
      conv1 = tf.layers.conv2d(inputs=self.input, filters=32,
                                     kernel_size=(8,8), strides=4, name='conv1',
                     kernel_initializer=tf.variance_scaling_initializer(scale=2))
      conv1_activated = tf.nn.relu(conv1)


      conv2 = tf.layers.conv2d(inputs=conv1_activated, filters=64,
                                     kernel_size=(4,4), strides=2, name='conv2',
                      kernel_initializer=tf.variance_scaling_initializer(scale=2))
      conv2_activated = tf.nn.relu(conv2)


      conv3 = tf.layers.conv2d(inputs=conv2_activated, filters=128,
                                     kernel_size=(3,3),strides=1, name='conv3',
                      kernel_initializer=tf.variance_scaling_initializer(scale=2))
      conv3_activated = tf.nn.relu(conv3)
      flat = tf.layers.flatten(conv3_activated)
      dense1 = tf.layers.dense(flat, units=self.fc1_dims,
                                     activation=tf.nn.relu,
                    kernel_initializer=tf.variance_scaling_initializer(scale=2))

      self.Q_values = tf.layers.dense(dense1, units=self.n_actions,
                    kernel_initializer=tf.variance_scaling_initializer(scale=2))

      #self.q = tf.reduce_sum(tf.multiply(self.Q_values, self.actions))

      self.loss = tf.reduce_mean(tf.square(self.Q_values - self.q_target))

      self.train_op = tf.train.AdamOptimizer(self.lr).minimize(self.loss)


# EXP



self.build_network()
This line calls the build_network method, which is likely responsible for constructing the neural network architecture. The implementation of this method is not shown in the provided code snippet.

python
Copy
self.sess.run(tf.global_variables_initializer())
This line initializes all the global variables (weights and biases) in the TensorFlow graph.

python
Copy
self.saver = tf.train.Saver()
This line creates a Saver object, which is used for saving and restoring the model's weights and variables.

python
Copy
self.checkpoint_file = os.path.join(chkpt_dir, 'deepqnet.ckpt')
This line constructs the path for the checkpoint file where the model's weights will be saved or loaded from. It joins the chkpt_dir path with the filename 'deepqnet.ckpt'.

python
Copy
self.params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)
This line retrieves all the trainable variables (weights and biases) from the TensorFlow graph that are associated with the scope self.name. These variables will be used during the training process.

In summary, this code defines a class deeepQNetwork that represents a deep Q-network (DQN) used in reinforcement learning. The class initializes the necessary parameters, creates a TensorFlow session, builds the neural network architecture (not shown), initializes variables, creates a saver for checkpointing, and retrieves the trainable variables from the TensorFlow graph. The DQN is likely used to approximate the Q-value function, which represents the expected cumulative reward for taking a particular action in a given state of the environment.

# WAY OF SAVING FILES

In [2]:
def load_checkpoint(self):
  print("...Loading checkpoint...")
  self.saver.restore(self.sess, self.checkpoint_file)

def save_checkpoint(self):
  print("...Saving checkpoint...")
  self.saver.save(self.sess, self.checkpoint_file)

agent

In [ ]:
class Agent(object):
    def __init__(self, alpha, gamma, mem_size, n_actions, epsilon, batch_size,
                 replace_target=10000, input_dims=(210,160,4),
                 q_next_dir='tmp/q_next', q_eval_dir='tmp/q_eval'):
        self.action_space = [i for i in range(n_actions)]
        self.n_actions = n_actions
        self.gamma = gamma
        self.mem_size = mem_size
        self.mem_cntr = 0
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.replace_target = replace_target
        self.q_next = DeepQNetwork(alpha, n_actions, input_dims=input_dims,
                                   name='q_next', chkpt_dir=q_next_dir)
        self.q_eval = DeepQNetwork(alpha, n_actions, input_dims=input_dims,
                                   name='q_eval', chkpt_dir=q_eval_dir)
        self.state_memory = np.zeros((self.mem_size, *input_dims))
        self.new_state_memory = np.zeros((self.mem_size, *input_dims))
        self.action_memory = np.zeros((self.mem_size, self.n_actions),
                                      dtype=np.int8)
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.int8)